In [1]:
import os
import re
import cv2
import sys
import numpy as np
from PIL import Image
from threading import *
from tqdm.auto import tqdm
from pyzbar.pyzbar import *

In [2]:
# <QR Format>
# 식당: NWTAVR-MA-202112-식당이름
# 직원: NWTAVR-MS-202112-4077-01

QR_HEADER = "NWTA(TA|TG|VR|VM)"		# 노원구청 교통행정과 등록팀 (내 QR인지 식별자 역할)
QR_CLASS_MA = "MA"					# 가맹점 = Member of Affiliation
QR_CLASS_MS = "MS"					# 직원 = Member of Staff
QR_YEAR = "2023"					# 대상년도 =========> 실행시 반드시 고칠 것
QR_MONTH = "02"						# 대상월   =========> 실행시 반드시 고칠 것
QR_YYYYMM = QR_YEAR + QR_MONTH		# "202112"
QR_SN = ["0"+str(i) if i < 10 else str(i) for i in range(1, 21)]    # ["01", "02", ... "19", "20"]

affil_list = ["노코(NOKO)", "어장촌생선구이", "횡성목장", "전주콩나루", "옛날칼국수", "칠리사이공", "도나한우", "북경(北京)", "명문식당", "일성스시", "새싹비빔밥", "파운드커피", "구내식당"]

code_list =  ["4080",    "4076",    "4079",    "4069",    "4078",    "4071",    "4068",    "4075",    "4077",    "4000" ]
name_list =  ["정미경",  "정성욱",  "강선임",  "모수지",  "송현우",  "신이은",  "이지은",  "김형수",  "김재형",  "한정민"]	# 직원명단 =========> 인사변동시 반드시 고칠 것
color_list = ["#E32636", "#FFA500", "#FF5511", "#0FF1C3", "#AB3ED8", "#AAEE00", "#1155FF", "#FCE205", "#119617", "#CC0000"]

name_by_code = {code_list[i] : name_list[i] for i in range(len(code_list))}			# {'4080':'전현호','4076':'정성욱',...,'4077':'김재형'}
color_by_code = {code_list[j] : color_list[j] for j in range(len(code_list))}		# {'4080':'#E32636','4076': '#FFA500',...,'4077':'#119617'}

# 교통행정과 모든 팀 포함한 정규식
# NWTA(TA|TG|VR|VM)-MA-202302-[0-9]{4}-[0-9]{2}

MA_pattern = re.compile(QR_HEADER + "-" + QR_CLASS_MA + "-" + QR_YYYYMM + "-" + ".+")							# NWTAVR-MA-202112-노코(NOKO)
MS_pattern = re.compile(QR_HEADER + "-" + QR_CLASS_MS + "-" + QR_YYYYMM + "-" + "[0-9]{4}" + "-" + "[0-9]{2}")	# NWTAVR-MS-202112-4077-01

In [3]:
# <장부 QR 동영상 읽기>
# 1) 가맹점별 동영상 1개 저장
# 2) 동영상 1개당 쓰레드 1개 실행

HOME_PATH = "C:/Dev/QRticket/"
MOV_PATH = HOME_PATH + QR_YYYYMM + "/"
OUT_PATH = HOME_PATH + QR_YYYYMM + "_Result" + "/"

file_list = os.listdir(MOV_PATH)		# 폴더 안의 모든 파일 = ['0.MOV', '1.MOV', ... '10.MOV', '11.MOV']
num_of_file = len(file_list)			# 동영상 파일 개수
NUM_OF_THREAD = num_of_file				# 식당 개수 = 동영상 파일 개수 = 쓰레드 개수

os.makedirs(OUT_PATH, exist_ok=True)
cap = [cv2.VideoCapture(os.path.join(MOV_PATH, file_list[i])) for i in range(num_of_file)]    # 동영상 파일 입력 ==> cap 리스트에 저장

In [4]:
# 파일읽기 테스트 : 파일을 읽을 수 없으면 False
for i in range(num_of_file):
	print(f"#{i+1:>2}/{num_of_file} cap[{i:>2}] file ready: ", cap[i].isOpened())

# 1/13 cap[ 0] file ready:  True
# 2/13 cap[ 1] file ready:  True
# 3/13 cap[ 2] file ready:  True
# 4/13 cap[ 3] file ready:  True
# 5/13 cap[ 4] file ready:  True
# 6/13 cap[ 5] file ready:  True
# 7/13 cap[ 6] file ready:  True
# 8/13 cap[ 7] file ready:  True
# 9/13 cap[ 8] file ready:  True
#10/13 cap[ 9] file ready:  True
#11/13 cap[10] file ready:  True
#12/13 cap[11] file ready:  True
#13/13 cap[12] file ready:  True


In [5]:
# MOV 파일 하나씩 QR검출 돌릴 쓰레드
class QRDetectorThread(Thread):
	
	# 초기화
	def __init__(self, cap=None, threadID=None):
		super(QRDetectorThread, self).__init__(daemon=True)
		
		self.cap = cap												# 입력 MOV 파일
		self.threadID = threadID									# 쓰레드 ID
		self.width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))			# 이번 MOV 파일의 width
		self.height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))		# 이번 MOV 파일의 height
		self.frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))	# 이번 MOV 파일의 frame 총수
		self.fps = cap.get(cv2.CAP_PROP_FPS)						# 이번 MOV 파일의 frame rate
		
		self.detected_MA_set = set()    # 식당QR (중복없음)
		self.detected_MS_set = set()    # 식권QR (중복없음)
		self.final_MA_name = ""			# 최종 식당 이름
		self.final_MS_dict = {}			# 최종 직원 명단 : {"전현호": [1, 4, 5], "정성욱": [2, 10], "박선녕": [],  "신상용": [],  ..., "김재형": [4, 12]}
        
        # DIVX 코덱으로 avi 파일 생성
		self.fourcc = cv2.VideoWriter_fourcc(*'DIVX')
		self.out = cv2.VideoWriter(f"{OUT_PATH}QR_result_{threadID}.avi", self.fourcc, self.fps, (self.width, self.height))
	
	# 쓰레드 실행
	def run(self):
		
		for frame_index in range(self.frame_count):

			# 읽어오지 못한 경우(success = False) ==> 쓰레드 종료
			success, frame = self.cap.read()
			if success == False:
				break

			# frame 1개에서 여러개의 QR 찾기
			QRs_in_frame = decode(frame, symbols=[ZBarSymbol.QRCODE])    # list
			num_of_QR = len(QRs_in_frame)
			# print(f"[전처리 전] 발견된 QR개수: {num_of_QR}")


			# 만약 하나도 발견되지 않았다면 다음 frame
			if num_of_QR == 0:
				# print("QR code not detected")
				continue


			# frame 1개에서 발견한 여러개 QR 중 1개 = eachQR
			for eachQR in QRs_in_frame:

				# QR 형식검증 : MS/MA패턴
				eachQR_data = eachQR.data.decode('utf-8')
				match_result_MA = MA_pattern.match(eachQR_data)
				match_result_MS = MS_pattern.match(eachQR_data)
				
				# QR이 가맹점도 직원도 아닌 경우 ==> 삭제 후 다음 eachQR
				if (match_result_MA is None) and (match_result_MS is None):
					QRs_in_frame.remove(eachQR)    # list에서 가장 index 앞서는 값 1개만 삭제
					continue
                        
				# 직원QR인 경우
				if match_result_MS is not None:
					self.detected_MS_set.add(match_result_MS.group())
					_header, _ms, _yyyymm, code, _sn = eachQR.data.decode('utf-8').split('-')
					color_hex_selected = color_by_code[code]
					color_selected = tuple(int(color_hex_selected[i:i+2], 16) for i in (5, 3, 1))	# HEX 문자열 -> BGR tuple값
                    
                # 가맹점QR인 경우
				if match_result_MA is not None:
					self.detected_MA_set.add(match_result_MA.group())
					color_selected = (0, 0, 255)    # 식당QR은 빨간색
                    
                # QR 테두리 선그리기 : QR 1개당 polygon 점의 개수(보통 4각형=4개)
				points = len(eachQR.polygon)

				# QR 1개당 경계선 그리기
				for point in range(points):
					next_point = (point+1) % points
					cv2.line(frame, tuple(eachQR.polygon[point]), tuple(eachQR.polygon[next_point]), color_selected, 5)    # (B,G,R), 굵기
				# ========== [안쪽 for loop 내부] frame 1개 중 QR 1개 처리 끝 ==========
                
            # ========== [안쪽 for loop 종료] frame 1개 중 QR 전체 처리 끝 ==========
			# num_of_QR_preprocessed = len(QRs_in_frame)
			# print("Frame  #{:3}/{:3} 처리 끝    ====>    전처리 전: {:2} \t 전처리 후: {:2}".format(frame_index+1, frame_count, num_of_QR, num_of_QR_preprocessed))
            
            # frame 1개 처리 끝 ==> 저장
			self.out.write(frame)

		self.out.release()	# 결과 파일 처리 끝
		self.cap.release()	# MOV 파일 읽기 끝
		# ========== [바깥 for loop 종료] MOV 파일의 모든 frame 처리 끝 ==========
		
		# ========== 검출된 QR자료 정리 시작 ==========
		MA_list = list(self.detected_MA_set)
		MA_list = sorted(MA_list)
		MS_list = list(self.detected_MS_set)
		MS_list = sorted(MS_list)
		
		# 가맹점 QR이 1개도 없으면 오류, 1개뿐이면 최종 저장
		if len(MA_list) != 1:
			print("[오류] QR 검출된 가맹점이 없거나, 2개 이상입니다.")
			sys.exit(0)
		else:
			self.final_MA_name = MA_list[0].split('-')[3]		# ['NWTAVR-MA-202112-새싹비빔밥']
		
		# 직원 QR은 하나도 없을 수도 있음 ==> 알림만 표시 (데이터 형태는 유지)
		if len(MS_list) == 0:
			print(f"[알림] 현재 쓰레드({self.threadID})에서 검출된 직원 QR이 없습니다.")

		# 직원 순서대로 final_MS_dict 초기화 : python 3.7부터 dict = ordered
		for _code in code_list:
			self.final_MS_dict[name_by_code[_code]] = []

		# QR 있는 직원만 final_MS_dict에 SN 채워넣기
		for eachStaff in MS_list:
			_header, _ms, _yyyymm, code, sn = eachStaff.split("-")
			self.final_MS_dict[name_by_code[code]].append(sn)
			
		
		print(f"[쓰레드 종료] ID: {self.threadID:>2}")
		#============================ 쓰레드 run() 처리 끝 ======================================

In [6]:
# 쓰레드 생성 + 시작
QR_workers = []
for i in range(NUM_OF_THREAD):
	QR_workers.append(QRDetectorThread(cap[i], i))
	QR_workers[i].start()

  0%|          | 0/979 [00:00<?, ?it/s]

  0%|          | 0/574 [00:00<?, ?it/s]

  0%|          | 0/198 [00:00<?, ?it/s]

  0%|          | 0/329 [00:00<?, ?it/s]

  0%|          | 0/389 [00:00<?, ?it/s]

...

  0%|          | 0/390 [00:00<?, ?it/s]

  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/162 [00:00<?, ?it/s]

  0%|          | 0/196 [00:00<?, ?it/s]

  0%|          | 0/421 [00:00<?, ?it/s]

In [7]:
# 최종 종합 통계를 위한 2개의 변수
# final_MA_list : 식당 이름의 리스트
# final_MS_list : 식당별 1개의 dict가 순서대로 저장된 리스트

final_MA_list = []
final_MS_list = []

for i in range(NUM_OF_THREAD):
	final_MA_list.append(QR_workers[i].final_MA_name)
	final_MS_list.append(QR_workers[i].final_MS_dict)

In [8]:
print(final_MA_list)
print(final_MS_list)

['노코(NOKO)', '어장촌생선구이', '횡성목장', '전주콩나루', '옛날칼국수', '칠리사이공', '도나한우', '북경(北京)', '명문식당', '일성스시', '새싹비빔밥', '파운드커피', '구내식당']
[{'정미경': [], '정성욱': [], '강선임': [], '모수지': ['04', '05'], '송현우': [], '신이은': [], '이지은': [], '김형수': [], '김재형': [], '한정민': []}, {'정미경': [], '정성욱': ['01', '12', '19'], '강선임': [], '모수지': [], '송현우': ['06', '12'], '신이은': [], '이지은': ['14', '15'], '김형수': ['03', '09'], '김재형': ['11'], '한정민': []}, {'정미경': [], '정성욱': ['06', '07', '08', '11', '14', '16'], '강선임': ['03', '08', '13', '18'], '모수지': ['01'], '송현우': ['01', '07', '16'], '신이은': ['01'], '이지은': ['01', '06', '19', '20'], '김형수': ['02', '07'], '김재형': ['01', '02', '06', '12', '17'], '한정민': []}, {'정미경': [], '정성욱': ['02', '05', '10', '13', '15', '17', '20'], '강선임': ['12', '17', '19'], '모수지': ['06'], '송현우': [], '신이은': [], '이지은': [], '김형수': ['05', '06'], '김재형': [], '한정민': []}, {'정미경': [], '정성욱': [], '강선임': [], '모수지': [], '송현우': [], '신이은': [], '이지은': ['02', '03', '04', '05', '07', '08', '11', '12', '13', '16', '17', '18'], '김형수': [], '김재

## 동영상 파일 QR처리 끝

## Pandas 데이터프레임에 최종 입력 시작

In [9]:
import pandas as pd

column_list = affil_list
column_list.insert(0, '직원성명')

df = pd.DataFrame(columns=column_list)

df['직원성명'] = name_list
df = df.set_index('직원성명')

df

,노코(NOKO),어장촌생선구이,횡성목장,전주콩나루,옛날칼국수,칠리사이공,도나한우,북경(北京),명문식당,일성스시,새싹비빔밥,파운드커피,구내식당
직원성명,,,,,,,,,,,,,
정미경,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
정성욱,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
강선임,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
모수지,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
송현우,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
신이은,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
이지은,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
김형수,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
김재형,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# 식당별로 테이블 채워넣기

for col_name, val_dict in zip(final_MA_list, final_MS_list):
	df[col_name] = list(map(len, val_dict.values()))
df

,노코(NOKO),어장촌생선구이,횡성목장,전주콩나루,옛날칼국수,칠리사이공,도나한우,북경(北京),명문식당,일성스시,새싹비빔밥,파운드커피,구내식당
직원성명,,,,,,,,,,,,,
정미경,0,0,0,0,0,0,2,5,0,0,0,0,7
정성욱,0,3,6,7,0,0,0,0,1,0,2,1,0
강선임,0,0,4,3,0,0,7,0,0,0,0,0,6
모수지,2,0,1,1,0,2,0,0,0,0,0,0,5
송현우,0,2,3,0,0,0,0,2,0,0,0,0,0
신이은,0,0,1,0,0,1,0,0,0,0,0,0,9
이지은,0,2,4,0,12,0,0,0,0,0,0,2,0
김형수,0,2,2,2,0,0,1,2,1,0,1,0,0
김재형,0,1,5,0,0,0,0,2,1,0,0,0,0


In [11]:
df2 = df.T
df2

직원성명,정미경,정성욱,강선임,모수지,송현우,신이은,이지은,김형수,김재형,한정민
노코(NOKO),0,0,0,2,0,0,0,0,0,0
어장촌생선구이,0,3,0,0,2,0,2,2,1,0
횡성목장,0,6,4,1,3,1,4,2,5,0
전주콩나루,0,7,3,1,0,0,0,2,0,0
옛날칼국수,0,0,0,0,0,0,12,0,0,0
칠리사이공,0,0,0,2,0,1,0,0,0,0
도나한우,2,0,7,0,0,0,0,1,0,0
북경(北京),5,0,0,0,2,0,0,2,2,0
명문식당,0,1,0,0,0,0,0,1,1,0
일성스시,0,0,0,0,0,0,0,0,0,0


In [12]:
df2.loc['<<직원별 식권 총계>>'] = df2.sum(axis=0)
df2

직원성명,정미경,정성욱,강선임,모수지,송현우,신이은,이지은,김형수,김재형,한정민
노코(NOKO),0,0,0,2,0,0,0,0,0,0
어장촌생선구이,0,3,0,0,2,0,2,2,1,0
횡성목장,0,6,4,1,3,1,4,2,5,0
전주콩나루,0,7,3,1,0,0,0,2,0,0
옛날칼국수,0,0,0,0,0,0,12,0,0,0
칠리사이공,0,0,0,2,0,1,0,0,0,0
도나한우,2,0,7,0,0,0,0,1,0,0
북경(北京),5,0,0,0,2,0,0,2,2,0
명문식당,0,1,0,0,0,0,0,1,1,0
일성스시,0,0,0,0,0,0,0,0,0,0


In [13]:
df2['<<식당별 식권 개수>>'] = df2.sum(axis=1)
df2

직원성명,정미경,정성욱,강선임,모수지,송현우,신이은,이지은,김형수,김재형,한정민,<<식당별 식권 개수>>
노코(NOKO),0,0,0,2,0,0,0,0,0,0,2
어장촌생선구이,0,3,0,0,2,0,2,2,1,0,10
횡성목장,0,6,4,1,3,1,4,2,5,0,26
전주콩나루,0,7,3,1,0,0,0,2,0,0,13
옛날칼국수,0,0,0,0,0,0,12,0,0,0,12
칠리사이공,0,0,0,2,0,1,0,0,0,0,3
도나한우,2,0,7,0,0,0,0,1,0,0,10
북경(北京),5,0,0,0,2,0,0,2,2,0,11
명문식당,0,1,0,0,0,0,0,1,1,0,3
일성스시,0,0,0,0,0,0,0,0,0,0,0


In [14]:
df2.to_excel(f"{QR_YYYYMM} 식권 사용내역.xlsx")

## 식권 처리 끝 !!